This file is just to test the connection between locally installed mistrial with ollama and python


In [3]:
import ollama

# Run a simple prompt
response = ollama.chat(model='mistral', messages=[
  {'role': 'user', 'content': 'What is the capital of France?'}
])

print(response['message']['content'])


 The capital of France is Paris.


A bit detailed example on how to use LLM for batch inputs

In [4]:
import ollama
import time
import json

# --- Step 1: Define your batch inputs with slight variations ---
base_prompt = "Summarize the importance of {} in human life."
keywords = ["patience", "gratitude", "forgiveness", "hope", "resilience"]

# Create structured input list
batch_inputs = [base_prompt.format(word) for word in keywords]

# --- Step 2: Store results ---
results = []

# --- Step 3: Loop through batch and query Ollama ---
for idx, prompt in enumerate(batch_inputs):
    print(f"\n[Prompt {idx+1}] -> {prompt}")
    start_time = time.time()

    response = ollama.chat(
        model='mistral',
        messages=[
            {'role': 'user', 'content': prompt}
        ]
    )

    end_time = time.time()
    elapsed = round(end_time - start_time, 2)

    structured = {
        "prompt": prompt,
        "response": response['message']['content'],
        "response_time_seconds": elapsed
    }

    results.append(structured)

# --- Step 4: Print final results in JSON format ---
print("\n=== Final Results ===")
print(json.dumps(results, indent=2))



[Prompt 1] -> Summarize the importance of patience in human life.

[Prompt 2] -> Summarize the importance of gratitude in human life.

[Prompt 3] -> Summarize the importance of forgiveness in human life.

[Prompt 4] -> Summarize the importance of hope in human life.

[Prompt 5] -> Summarize the importance of resilience in human life.

=== Final Results ===
[
  {
    "prompt": "Summarize the importance of patience in human life.",
    "response": " Patience plays a crucial role in human life, offering numerous benefits and contributions to personal growth, relationships, and overall well-being. Here are some key aspects that highlight the importance of patience:\n\n1. Emotional Regulation: Patience helps individuals manage their emotions more effectively, reducing stress levels and promoting a positive mindset. When faced with challenging situations or setbacks, patience allows people to remain calm and composed, fostering resilience and adaptability.\n\n2. Decision-Making: Being patie

Parallel execution of prompts

In [5]:
import ollama
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- Step 1: Prepare inputs ---
base_prompt = "Summarize the importance of {} in human life."
keywords = ["patience", "gratitude", "forgiveness", "hope", "resilience", "trust", "kindness", "humility", "courage", "empathy"]
batch_inputs = [base_prompt.format(word) for word in keywords]

# --- Step 2: Define function for threaded execution ---
def query_ollama(prompt):
    start_time = time.time()
    try:
        response = ollama.chat(
            model='mistral',
            messages=[{'role': 'user', 'content': prompt}]
        )
        elapsed = round(time.time() - start_time, 2)
        return {
            "prompt": prompt,
            "response": response['message']['content'],
            "response_time_seconds": elapsed
        }
    except Exception as e:
        return {
            "prompt": prompt,
            "response": f"Error: {str(e)}",
            "response_time_seconds": None
        }

# --- Step 3: Execute in parallel ---
results = []
max_threads = 4  # Set based on your CPU capability

with ThreadPoolExecutor(max_workers=max_threads) as executor:
    future_to_prompt = {executor.submit(query_ollama, prompt): prompt for prompt in batch_inputs}

    for future in as_completed(future_to_prompt):
        result = future.result()
        print(f"[Done] Prompt: {result['prompt']} | Time: {result['response_time_seconds']}s")
        results.append(result)

# --- Step 4: Output structured results ---
print("\n=== Final Results (JSON) ===")
print(json.dumps(results, indent=2))

# --- Optional: Save to file ---
with open("parallel_mistral_results.json", "w") as f:
    json.dump(results, f, indent=2)


[Done] Prompt: Summarize the importance of hope in human life. | Time: 12.22s
[Done] Prompt: Summarize the importance of gratitude in human life. | Time: 21.9s
[Done] Prompt: Summarize the importance of patience in human life. | Time: 31.03s
[Done] Prompt: Summarize the importance of resilience in human life. | Time: 19.88s
[Done] Prompt: Summarize the importance of forgiveness in human life. | Time: 38.93s
[Done] Prompt: Summarize the importance of trust in human life. | Time: 22.55s
[Done] Prompt: Summarize the importance of courage in human life. | Time: 11.25s
[Done] Prompt: Summarize the importance of empathy in human life. | Time: 8.92s
[Done] Prompt: Summarize the importance of kindness in human life. | Time: 28.7s
[Done] Prompt: Summarize the importance of humility in human life. | Time: 30.86s

=== Final Results (JSON) ===
[
  {
    "prompt": "Summarize the importance of hope in human life.",
    "response": " Hope plays a crucial role in human life, serving as an essential ps